<a href="https://colab.research.google.com/github/khacminh181/NER-Recognition/blob/master/NER_CRF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import files
uploaded = files.upload()

Saving DataNer.txt to DataNer.txt


In [4]:
with open('DataNer.txt','r',encoding='utf-8') as f:
    text = f.read()
    list_senteces = list()
    sentence=list()
    for line in text.splitlines():
        if line!='':
            word = line.split()
            row = (word[0], word[1], word[4])
            sentence.append(row)
        else:
            list_senteces.append(sentence)
            sentence = list()
# list_senteces[0:4]
# format data into a list of lists, every childlist is a sentences, includes tuples - word
list_senteces

[[('cho', 'E', 'O'),
  ('em', 'N', 'O'),
  ('hỏi', 'V', 'O'),
  ('placement', 'N', 'O'),
  ('test', 'A', 'O'),
  ('thi', 'V', 'O'),
  ('những', 'L', 'O'),
  ('nôi', 'N', 'O'),
  ('dung', 'N', 'O'),
  ('nào', 'P', 'O')],
 [('placement', 'N', 'O'),
  ('test', 'A', 'O'),
  ('có', 'V', 'O'),
  ('mấy', 'L', 'O'),
  ('phần', 'N', 'O'),
  ('tất_cả', 'P', 'O'),
  ('ạ', 'I', 'O')],
 [('Trong', 'E', 'O'),
  ('bài', 'N', 'O'),
  ('thi', 'V', 'O'),
  ('tiếng', 'N', 'B-Subject_Name'),
  ('anh', 'Nc', 'I-Subject_Name'),
  ('kỹ_năng', 'N', 'O'),
  ('nói', 'V', 'O'),
  ('chiếm', 'V', 'O'),
  ('bao_nhiêu', 'P', 'O'),
  ('điểm', 'N', 'O'),
  ('ạ', 'I', 'O'),
  ('?', 'CH', 'O')],
 [('thời_gian', 'N', 'O'),
  ('cho', 'E', 'O'),
  ('kỹ_năng', 'N', 'O'),
  ('nói', 'V', 'O'),
  ('là', 'C', 'O'),
  ('bao_lâu', 'P', 'O'),
  ('ạ', 'I', 'O')],
 [('hình_thức', 'N', 'O'),
  ('thi', 'V', 'O'),
  ('tiếng', 'N', 'B-Subject_Name'),
  ('anh', 'Nc', 'I-Subject_Name'),
  ('là', 'V', 'O'),
  ('gì', 'P', 'O'),
  ('ạ', 'T',

### Feature Extraction
Next, we extract more features (word parts, simplified POS tags, lower/title/upper flags, features of nearby words) and convert them to sklearn-crfsuite format — each sentence should be converted to a list of dicts

In [0]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i>0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1: word.lower()': word1.lower(),
            '-1: word.istitle()': word1.istitle(),
            '-1: word.isupper()': word1.isupper(),
            '-1: postag': postag1,
            '-1: postag[:2]': postag1[:2],
        })
    else:
        features['BOS']=True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1: word.lower()': word1.lower(),
            '+1: word.istitle()': word1.istitle(),
            '+1: word.isupper()': word1.isupper(),
            '+1: postag': postag1,
            '+1: postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features

def sent2features(sent):
    return [word2features(sent,i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]


In [8]:
from sklearn.model_selection import train_test_split
X = [sent2features(s) for s in list_senteces]
y = [sent2labels(s) for s in list_senteces]

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=0)
X_train[0]

[{'+1: postag': 'N',
  '+1: postag[:2]': 'N',
  '+1: word.istitle()': False,
  '+1: word.isupper()': False,
  '+1: word.lower()': 'em',
  'BOS': True,
  'bias': 1.0,
  'postag': 'E',
  'postag[:2]': 'E',
  'word.isdigit()': False,
  'word.istitle()': False,
  'word.isupper()': False,
  'word.lower()': 'cho',
  'word[-3:]': 'cho'},
 {'+1: postag': 'V',
  '+1: postag[:2]': 'V',
  '+1: word.istitle()': False,
  '+1: word.isupper()': False,
  '+1: word.lower()': 'hỏi',
  '-1: postag': 'E',
  '-1: postag[:2]': 'E',
  '-1: word.istitle()': False,
  '-1: word.isupper()': False,
  '-1: word.lower()': 'cho',
  'bias': 1.0,
  'postag': 'N',
  'postag[:2]': 'N',
  'word.isdigit()': False,
  'word.istitle()': False,
  'word.isupper()': False,
  'word.lower()': 'em',
  'word[-3:]': 'em'},
 {'+1: postag': 'Nb',
  '+1: postag[:2]': 'Nb',
  '+1: word.istitle()': False,
  '+1: word.isupper()': False,
  '+1: word.lower()': 'email',
  '-1: postag': 'N',
  '-1: postag[:2]': 'N',
  '-1: word.istitle()': Fa

### Training


In [14]:
import sklearn_crfsuite
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1 = 0.1,
    c2 = 0.1,
    max_iterations = 100,
    all_possible_transitions=False,
)
crf.fit(X_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=False,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

In [13]:
!pip install sklearn_crfsuite



     |████████████████████████████████| 747kB 3.9MB/s 


### Evaluation

In [18]:
labels = list(crf.classes_)
labels.remove('O')
labels

['B-Teacher_Name',
 'I-Teacher_Name',
 'B-Major_Name',
 'B-Subject_Name',
 'I-Subject_Name',
 'B-Major_Mode',
 'I-Major_Mode',
 'B-Scholar_Name',
 'I-Scholar_Name',
 'B-Docs_Name',
 'I-Docs_Name',
 'B-Uni_Name',
 'I-Uni_Name',
 'B-datetime',
 'I-datetime',
 'I-Major_Name',
 'B-number',
 'B-Campus_Name',
 'I-Campus_Name',
 'B-Admission_Type',
 'I-Admission_Type',
 'B-K_years',
 'I-K_years',
 'B-duration',
 'I-duration',
 'B-Dept_Name',
 'I-Dept_Name',
 'B-location',
 'I-number']

In [20]:
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred,
                      average='weighted', labels=labels)

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)


0.892855682537868

In [21]:
# group B and I results
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

                  precision    recall  f1-score   support

B-Admission_Type      0.911     0.872     0.891        47
I-Admission_Type      0.829     0.840     0.834        81
   B-Campus_Name      1.000     0.750     0.857        20
   I-Campus_Name      1.000     0.684     0.813        19
     B-Dept_Name      1.000     0.714     0.833         7
     I-Dept_Name      1.000     0.778     0.875         9
     B-Docs_Name      0.929     0.667     0.776        39
     I-Docs_Name      0.870     0.720     0.788        93
       B-K_years      1.000     0.571     0.727         7
       I-K_years      1.000     0.364     0.533        11
    B-Major_Mode      0.913     0.857     0.884        49
    I-Major_Mode      0.988     0.814     0.893        97
    B-Major_Name      0.975     0.902     0.937       132
    I-Major_Name      0.949     0.925     0.937       200
  B-Scholar_Name      0.939     0.939     0.939        49
  I-Scholar_Name      0.953     0.821     0.882       123
  B-Subject_N

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [24]:
from seqeval.metrics import classification_report
print(classification_report(y_test,y_pred))


                precision    recall  f1-score   support

      datetime       0.94      0.92      0.93        99
      Uni_Name       0.91      0.89      0.90       185
  Scholar_Name       0.88      0.88      0.88        49
    Major_Mode       0.87      0.82      0.84        49
    Major_Name       0.93      0.86      0.89       132
      duration       0.63      0.80      0.71        15
     Docs_Name       0.79      0.56      0.66        39
       K_years       1.00      0.57      0.73         7
        number       0.97      0.80      0.88        44
     Dept_Name       0.80      0.57      0.67         7
Admission_Type       0.91      0.87      0.89        47
   Campus_Name       0.93      0.70      0.80        20
  Subject_Name       0.96      0.92      0.94        24
  Teacher_Name       1.00      1.00      1.00         5

     micro avg       0.90      0.84      0.87       722
     macro avg       0.90      0.84      0.87       722



In [23]:
!pip install seqeval

  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7424 sha256=c361577cf31cc54e6326a4e5b644e68f4dcec3a67ced3fdb3b809a46761f7255
  Stored in directory: /root/.cache/pip/wheels/4f/32/0a/df3b340a82583566975377d65e724895b3fad101a3fb729f68
Successfully built seqeval
